In [1]:
import itertools
import pandas as pd
import cobra.flux_analysis
from cobra import Metabolite
import time
import numpy as np
from src.mp_functions import combinations_subset, parallelize_dataframe, knockout_FBA, knockout_FBA_w_tasks, knockout_FBA_simple

from functools import partial
from src.FBA_scripts.met_task_functions import constrain_model, read_tasks


In [6]:
# Single model
model_file_path = 'C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/GTEx/liver.xml'
model = cobra.io.read_sbml_model(model_file_path)

In [84]:
# Models for tasks
model_list = constrain_model(model_file_path, ALLMETSIN_OUT=True)

Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmp_v3f30r_.lp
Reading time = 0.07 seconds
: 5788 rows, 15892 columns, 69376 nonzeros
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpdcskp4v3.lp
Reading time = 0.07 seconds
: 5788 rows, 15892 columns, 69376 nonzeros


In [85]:
task_list = read_tasks('C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/essential_tasks.tsv', model_list)
# Constrain model
task = task_list[-1]
for subset in [task[0], task[1]]:
    for rx in subset:
        if rx == 'ALLMETSIN':
            # Adds boundary metabolites for other reactions when ALLMETSIN is used
            for r in subset[1:]:
                for m2 in r.metabolites:
                    for r2 in m2.reactions:
                        if r2.boundary and r2.id != r.id:
                            # Could also just remove the reactions, or set them 0, 0
                            r2.add_metabolites({Metabolite(
                                                m2.id[:-4] + 'x[x]',
                                                formula=m2.formula,
                                                name=' '.join(m2.name.split(' ')[:-1]) + ' [Boundary]',
                                                compartment='x'): 1})
            continue
        model_list[3].add_reaction(rx)


In [16]:
# FBA + knockouts

res_list = []

In [17]:
res_list.append(round(knockout_FBA_simple(model, []), 3))
res_list.append(round(knockout_FBA_simple(model, ['ENSG00000134538']), 3))

In [114]:
model = model_list[3]
with model:
    # Bilirubin sink
    model.reactions.get_by_id('HMR_9273').lower_bound = -1000
    model.reactions.get_by_id('HMR_9273').upper_bound = -249

    # Bilirubin-Bisglucuronoside sink
    model.reactions.get_by_id('HMR_9271').lower_bound = 0
    model.reactions.get_by_id('HMR_9271').upper_bound = 0

    # Bilirubin-monoglucuronoside sink
    model.reactions.get_by_id('HMR_9272').lower_bound = 10
    model.reactions.get_by_id('HMR_9272').upper_bound = 1000

    # Heme synthesis
    model.reactions.get_by_id('HMR_4758').lower_bound = 0
    model.reactions.get_by_id('HMR_4758').upper_bound = 1

    # Oxygen
    model.reactions.get_by_id('HMR_9048').lower_bound = -10
    # Glucose
    model.reactions.get_by_id('HMR_9034').lower_bound = -6

    # Threonine
    model.reactions.get_by_id('HMR_9044').lower_bound = -1

    #model.objective = 'HMR_9271'
    #model.objective_direction = 'max'

    print(knockout_FBA_simple(model, []))
    print(knockout_FBA_simple(model, ['ENSG00000134538', 'ENSG00000184999', 'ENSG00000103222']))

2.1839945137303576
2.1839945137303576


In [7]:
res_list_lung = res_list

[82.17122442635676, 81.6253209998778]


In [18]:
print('Lung results: ' + str(res_list_lung))
print(res_list)


Lung results: [82.17122442635676, 81.6253209998778]
[111.339, 111.339]
